In [1]:
import pandas as pd

Data exploration

In [2]:
df = pd.read_csv('pairing_food_combined.csv')

In [3]:
df.head(5)

,Pairing Food,Product,Grape,Popularity,Critics' Score,Avg. Price kr / 750ml
0,beef and venison,"Chateau Mouton Rothschild, Pauillac, France",Bordeaux Blend Red,1st,96 / 100,"7,926"
1,beef and venison,"Chateau Lafite Rothschild, Pauillac, France",Bordeaux Blend Red,3rd,96 / 100,"10,564"
2,beef and venison,"Petrus, Pomerol, France",Merlot,4th,96 / 100,"47,971"
3,beef and venison,"Tenuta San Guido Sassicaia Bolgheri, Tuscany, ...",Cabernet Franc - Cabernet Sauvignon,5th,95 / 100,"3,930"
4,beef and venison,"Chateau Margaux, Margaux, France",Bordeaux Blend Red,6th,96 / 100,"8,419"


In [4]:
unique_grapes = df['Grape'].unique()
unique_grapes
length_grapes = len(unique_grapes)
length_grapes
#331 unique grapes, so maybe we can use that to predict? 
#-> We can make a model that predicts for the grape kind which pairing food works best
#For example: It would say for (new) product x, the best pairing food is y, based on the grape z being used in the product

331

In [5]:
unique_products = df['Product'].unique()
unique_products
length_products = len(unique_products)
length_products
#there are 9741 unique products, out of 9750 total entries in the dataframe (so almost all of them are unique)

9741

# Data cleaning

So the grapes are spanning across multiple categories of food, while each wine mostly fits to just one category. 
maybe we can try to combine grape with country, and see if we find some more combinations there
-> so: a french Merlot works for beef, but a spanish Merlot works for some cheese?

lets start by trying to separate the "products" column into columns "wine", "region", and "country".

In [6]:
# Split the values in "column_name" by commas and expand them into separate columns
df[['Wine', 'Region', 'Country']] = df['Product'].str.split(',',n = 2, expand=True)

# Drop the original "column_name" column if needed
df.drop(columns=['Product'], inplace=True)

df.head(5)

,Pairing Food,Grape,Popularity,Critics' Score,Avg. Price kr / 750ml,Wine,Region,Country
0,beef and venison,Bordeaux Blend Red,1st,96 / 100,"7,926",Chateau Mouton Rothschild,Pauillac,France
1,beef and venison,Bordeaux Blend Red,3rd,96 / 100,"10,564",Chateau Lafite Rothschild,Pauillac,France
2,beef and venison,Merlot,4th,96 / 100,"47,971",Petrus,Pomerol,France
3,beef and venison,Cabernet Franc - Cabernet Sauvignon,5th,95 / 100,"3,930",Tenuta San Guido Sassicaia Bolgheri,Tuscany,Italy
4,beef and venison,Bordeaux Blend Red,6th,96 / 100,"8,419",Chateau Margaux,Margaux,France


In [8]:
df.shape

(9750, 8)

In [7]:
unique_countries = df['Country'].unique()
unique_countries
#we can see there are 45 unique entries in the countries column.
#there are three from italy: "sicily, Italy" and "veneto, Italy" and "Tuscany, Italy"
#-> as well as of course just "italy".
#there are also three entries from France: "France", "Vin de France", and "Vin de france". We should make those all the same


array([' France', ' Italy', ' USA', None, ' Spain', ' Chile',
       ' Argentina', ' Australia', ' South Africa', ' Portugal',
       ' New Zealand', ' Tuscany, Italy', ' Canada', ' Germany', ' China',
       ' Romania', ' Austria', ' Georgian Republic', ' Moldova',
       ' Israel', ' Sweden', ' UK', ' England', ' Switzerland',
       ' Hungary', ' Greece', ' Czech Republic', ' Brazil', ' Russia',
       ' Japan', ' Vin de France', ' Montenegro', ' Monterey, USA',
       ' Uruguay', ' Vin de france', ' Slovenia', ' Croatia', ' India',
       ' Cyprus', ' Turkey', ' Marlborough, New Zealand', ' Lebanon',
       ' Sicily, Italy', ' Veneto, Italy', ' Slovakia'], dtype=object)

Time to clean the code for the "country" column

In [9]:
#first, showing all the columns where the country is Tuscany, italy
condition = df['Country'] == ' Tuscany, Italy' 
filtered_df = df[condition]
filtered_df
    #just one data entry, and it shows that something went wrong with the separating of columns since the wine had an additional
        #comma in its name. We will drop this wine for consistency.
df = df[df['Country'] != ' Tuscany, Italy']

#next, showing columns where the country is "Sicily, Italy"
condition = df['Country'] == ' Sicily, Italy'
filtered_df = df[condition]
filtered_df
    #again, just a single entry where something likely went wrong with splitting. We will drop this entry as well
df = df[df['Country'] != ' Sicily, Italy']

#lastly, showing columns where the country is "Veneto, Italy"
condition = df['Country'] == ' Veneto, Italy'
filtered_df = df[condition]
filtered_df
    #just one entry, so dropping that as well
df = df[df['Country'] != ' Veneto, Italy']

#showing columns where the country is marlborough, NZ
condition = df['Country'] == ' Marlborough, New Zealand'
filtered_df = df[condition]
filtered_df
    #again one entry, but region is correct. So, renaming it
df['Country'].replace({' Marlborough, New Zealand': ' New Zealand'}, inplace=True)

#next, solving the issue of three france countries
df['Country'].replace({' Vin de France': ' France', ' Vin de france': ' France'}, inplace=True)

#next, looking at the difference between 'UK' and 'England'
condition = df['Country'] == ' UK'
filtered_df = df[condition]
filtered_df
    #UK has only two entries, where region is set to england and wine is Bulmers cider
condition = df['Country'] == ' England'
filtered_df = df[condition]
filtered_df
    #England has a bunch of entries with actual regions. To make data analysis easier, we are changing UK to england.
df['Country'].replace({' UK': ' England'}, inplace=True)

#finally, looking at ' Monterey, USA'
condition = df['Country'] == ' Monterey, USA'
filtered_df = df[condition]
filtered_df
    #just one entry, with region of california. So, changing Monteray to just USA
df['Country'].replace({' Monterey, USA': ' USA'}, inplace=True)

Next, time to check if the country column is cleaned now:

In [10]:
unique_countries = df['Country'].unique()
unique_countries
#looks much better now!
length_countries = len(unique_countries)
length_countries
#37 unique countries in the dataset

37

Now let's see how many different wines there are in the dataset now that we split off country and region.

In [11]:
unique_wines = df['Wine'].unique()
unique_wines
length_wines = len(unique_wines)
length_wines
#still 9734 unique wines in the dataset, thus mostly all unique wines.

9734

In [12]:
df

,Pairing Food,Grape,Popularity,Critics' Score,Avg. Price kr / 750ml,Wine,Region,Country
0,beef and venison,Bordeaux Blend Red,1st,96 / 100,"7,926",Chateau Mouton Rothschild,Pauillac,France
1,beef and venison,Bordeaux Blend Red,3rd,96 / 100,"10,564",Chateau Lafite Rothschild,Pauillac,France
2,beef and venison,Merlot,4th,96 / 100,"47,971",Petrus,Pomerol,France
3,beef and venison,Cabernet Franc - Cabernet Sauvignon,5th,95 / 100,"3,930",Tenuta San Guido Sassicaia Bolgheri,Tuscany,Italy
4,beef and venison,Bordeaux Blend Red,6th,96 / 100,"8,419",Chateau Margaux,Margaux,France
...,...,...,...,...,...,...,...,...
9745,white fish,Pinot Grigio,"22,922nd",81 / 100,160,Torresella Pinot Grigio Veneto IGT,Italy,None
9746,white fish,Verdicchio,"23,113th",89 / 100,213,Allegrini 'Oasi Mantellina' Lugana,Lombardy,Italy
9747,white fish,Biancolella,"23,113th",91 / 100,226,Casa d'Ambra Frassitelli Biancolella Ischia,Campania,Italy
9748,white fish,Soave Blend,"23,113th",85 / 100,80,Bolla Soave Classico,Veneto,Italy


In [13]:
df.to_csv('pairing_food.csv', index=False)